In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
try:
    print("Repo path:", repo_path)
except:
    os.chdir("../")
    repo_path = os.getcwd()
    print("Repo path:", repo_path)

In [ ]:
data_path = "data"
use_dir = "use"
glove_dir = "glove"

os.chdir(repo_path)

In [ ]:
os.listdir(data_path)

In [ ]:
df = pd.read_csv(os.path.join(data_path, "train.csv"))

### Is the dataset balanced?

In [ ]:
label1 = df["target"][df["target"] == 1].count()
label0 = df["target"][df["target"] == 0].count()
print("label 1:", (label1 /(label1 + label0)))
print("label 0:", (label0 /(label1 + label0)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
# Data to plot
labels = 'Disastrous', 'Normal'
sizes = [429, 571]
colors = ['Red', 'green']
explode = (0.1, 0)#, 0, 0)  # explode 1st slice

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,
autopct='%1.1f%%', shadow=True, startangle=140)

plt.axis('equal')
plt.show()

In [ ]:
df.head(10)

## Text data cleanning

In [ ]:
import nltk
import re
import string

ps = nltk.PorterStemmer()

def txt_cleanning(text, tokenize=False, stem=False):
    """ cleanning:
            lowercase
            remove numbers
            remove punctuation
            stem words
    """
    if not isinstance(text, list):
        # workaround in order to be able to re-run it
        text  = "".join([char.lower() for char in text ])
        text = re.sub('[0-9]+', '', text)
        text = text.split()
        out_text = []
        for word in text:
            if "\\x" in word:
                continue
            if "http" in word:
                out_text.append("http")
                continue
            if word == "ain't":
                out_text += ["is", "not"]
                continue
            if word == "won't":
                out_text += ["will", "not"]
                continue
            if word and word[0] in string.punctuation:
                word = word[1:]
            if word and word[-1] in string.punctuation:
                word = word[:-1]
            if len(word) > 4 and "n't" in word[-3:]:
                out_text.append(word[:-3])
                out_text.append("not")
            elif "'" in word:
                new_words = word.split("'")
                out_text += [new_words[0], "'" + new_words[1]]
            elif "-" in word:
                out_text += word.split("-")
            elif word:
                word = re.sub("[^a-z]+", "", word)
                out_text  += "".join([char if char.isalpha()
                                              else " " for char in word ]).split()
    out_text = [word.strip() for word in out_text if word.strip()]
    if stem:
        text = [ps.stem(word) for word in out_text]  # remove stopwords and stemming
    if not tokenize:
        out_text = " ".join([str(elem) for elem in out_text])
    return out_text

df['clean_text'] = df['text'].apply(lambda x: txt_cleanning(x))


### Train/Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_df, y_df = df["text"], df["target"]
# X_df, y_df = df["clean_text"], df["target"]

In [ ]:
ind=None
X_train, X_test, y_train, y_test = train_test_split(X_df[:ind], y_df[:ind], test_size=0.20, random_state=3)

In [ ]:
print("train:", X_train.shape)
print("test:", X_test.shape)

In [ ]:
label1 = y_train[y_train==1].count()
label0 = y_train[y_train==0].count()
print("label 1:", (label1 /(label1 + label0)))
print("label 0:", (label0 /(label1 + label0)))

In [ ]:
label1 = y_test[y_test==1].count()
label0 = y_test[y_test==0].count()
print("label 1:", (label1 /(label1 + label0)))
print("label 0:", (label0 /(label1 + label0)))

### USE

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from tqdm import tqdm

try:
    print("USE module:\n", use_module)
except:
    use_module = hub.load(use_dir)
    print("USE module:\n", use_module)

In [ ]:
def get_use(text, use_module=use_module):
    text = " ".join(text) if isinstance(text, list) else text
    text = [text.strip()]
    emb = use_module.signatures["response_encoder"](
        input=tf.constant(text),
        context=tf.constant(text))["outputs"].numpy()
    return emb

In [ ]:
test = "This is a test"
get_use(test, use_module).squeeze().shape

In [ ]:
X_train_use = []

for text in tqdm(X_train):
    X_train_use.append(get_use(text, use_module).squeeze())

In [ ]:
X_test_use = []

for text in tqdm(X_test):
    X_test_use.append(get_use(text, use_module).squeeze())

### TF-IDF

In [ ]:
# Create the vectorizer method
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vec = TfidfVectorizer(min_df=3, max_df=0.8, stop_words="english")

In [ ]:
tfidf_vec.fit(X_train)

In [ ]:
X_train_tfidf = tfidf_vec.transform(X_train).toarray()
X_test_tfidf = tfidf_vec.transform(X_test).toarray()

In [ ]:
X_train_tfidf.shape

## GloVe

In [ ]:
%%time
glove_voc = []
glove_weights = []
glove_dimensions = 25
glove_file = os.path.join(glove_dir, "glove.twitter.27B.{}d.txt".format(glove_dimensions))
with open(glove_file, 'r') as f:
    for line in f:
        values = line.split()
        if len(values[1:]) == glove_dimensions:
            glove_voc.append(values[0])
            glove_weights.append(values[1:])
print("GloVe embeddings imported.\nVector size: ")
print(len(glove_weights[-1]))
print()

In [ ]:
%%time
glove_weights = np.asarray(glove_weights, "float32")

In [ ]:
(glove_voc_length, glove_weights_length) = glove_weights.shape

## Naive Bayes classifier

In [ ]:
X_train_input = X_train_use
# X_train_input = X_train_tfidf
X_test_input = X_test_use
# X_test_input = X_test_tfidf

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_input, y_train)

In [ ]:
# Print out the model's accuracy
nb.score(X_test_input, y_test)

## Encode words

### Vocabulary to integer

In [ ]:
import string

class Vocabulary():
    def __init__(self):
        self.PAD = 0
        self.index2word = {self.PAD: "PAD"}
        self.word2index = {"PAD": self.PAD}
        self.size = 1
        self.oov = {}
        
    
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.size
            self.index2word[self.size] = word 
            self.size += 1
        return self.size-1
    
    def add_list(self, voc_list):
        if not isinstance(voc_list, list):
            voc_list = voc_list.split()
        for word in voc_list:
            self.add_word(word)
        return "1"

    def get_word(self, index):
        try:
            return self.index2word[index]
        except:
            return "Error: INDEX TOO BIG"

    def get_index(self, word):
        try:
            return self.word2index[word]
        except:
            if word not in self.oov:
                self.oov[word] = 1
            else:
                self.oov[word] += 1
            return -1
    def res_oov(self):
        self.oov = {}
        return "1"

In [ ]:
my_vocabulary = Vocabulary()
my_vocabulary.add_list(glove_voc)

In [ ]:
my_vocabulary.get_index("?")

In [ ]:
glove_voc[14]

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(0)

In [ ]:
embeddings = nn.Embedding(glove_voc_length+1, glove_weights_length, padding_idx=0)
embeddings.weight = nn.Parameter(torch.cat([torch.zeros(1, glove_dimensions), torch.tensor(glove_weights)], dim=0))
embeddings.weight.requires_grad = False

In [ ]:
inpt = torch.LongTensor([[glove_voc_length]])
embeddings(inpt) == torch.tensor(glove_weights[glove_voc_length-1])

### DataLoaders

In [ ]:
def sentence_to_index(lst, sentence_length, vocabulary):
    lst = lst if isinstance(lst, list) else lst.split()
    ret = [vocabulary.get_index(word.lower()) for word in lst]
    if len(ret) < sentence_length:
        ret += [0 for i in range(sentence_length - len(ret))]
    else:
        ret = ret[:sentence_length]
    return ret

In [ ]:
my_vocabulary.res_oov()
X = df["clean_text"].apply(lambda x: sentence_to_index(x, 15, my_vocabulary))
X

In [ ]:
my_vocabulary.oov

In [ ]:
def create_dataloader(X, y, vocabulary, batch_size=64, sentence_length=15):
    X = np.vstack(X.apply(lambda x: sentence_to_index(x, sentence_length, vocabulary)))
    dataset = TensorDataset(torch.from_numpy(X), torch.from_numpy(y.values))
    dataloader = DataLoader(dataset, shuffle=True, batch_size=batch_size)
    return dataloader

In [ ]:
train_dataloader = create_dataloader(X_train, y_train, my_vocabulary, batch_size=batch_size)
test_dataloader = create_dataloader(X_test, y_test, my_vocabulary, batch_size=batch_size)

### top OOV words 

In [ ]:
my_vocabulary.res_oov
df["text"].apply(lambda x: sentence_to_index(x, 15, my_vocabulary))

x = my_vocabulary.oov
x = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}

total = 0
for i,j in x.items():
    total += j
print(total)
sting = ""
t = 0
for word,count in x.items():
    if t == 100:
        break
    for i in range(count):
        sting += word + " "
    t += 1
list(x.keys())[:100]

In [ ]:
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

rcParams['figure.figsize'] = 13, 11
# Make fake dataset
height = list(x.values())[:30]
bars = list(x.keys())[:30]
y_pos = np.arange(len(bars))
 
# Create horizontal bars
plt.barh(y_pos, height)
 
# Create names on the y-axis
plt.yticks(y_pos, bars, fontsize=20)

# Show graphic
plt.show()
 

### top Words in cleaned dataset

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline


def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        max_words=300,
        max_font_size=40,
        margin=0,
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(df["clean_text"])

### Submit

In [ ]:
from src.models.tweet import TweetClassifier

In [ ]:
!ls

In [ ]:
lstm_tweet = TweetClassifier("lstm")
lstm_tweet.load("model_bin/lstm.pt")

In [ ]:
snn_tweet = TweetClassifier("snn")
snn_tweet.load("model_bin/snn.pt")

## Submit to Kaggle

### Read testing set

In [ ]:
df_test = pd.read_csv(os.path.join(data_path, "test.csv"))
df_test["clean_text"] = df_test["text"].apply(lambda x: txt_cleanning(x))
df_test.head()

In [ ]:
# submit_X = df_test["text"]
submit_X = df_test["clean_text"]

In [ ]:
# submit_X_tfidf = tfidf_vec.transform(submit_X).toarray()

In [ ]:
# submit_X_use = []

# for text in tqdm(submit_X):
#     submit_X_use.append(get_use(text, use_module).squeeze())

In [ ]:
submit_X_inputs = submit_X.values

In [ ]:
# submit_preds = lstm_tweet.predict(submit_X_inputs)

In [ ]:
submit_preds = snn_tweet.predict(submit_X_inputs)

In [ ]:
# submit_preds = nb.predict(submit_X_inputs)

In [ ]:
submit_preds = [int(x) for x in submit_preds]

In [ ]:
to_submit_df = pd.DataFrame(submit_preds, columns=["target"])

to_submit_df = pd.concat((df_test["id"], to_submit_df), axis = 1)

to_submit_df.to_csv("submission.csv", index=False)

In [ ]:
!kaggle competitions submit -c nlp-getting-started  -f submission.csv -m "Test"